Import the libraries

In [1]:
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate, KFold
from sklearn.preprocessing import MinMaxScaler
import warnings
import traceback

Import the data

In [2]:
# Paths to the pickle files
independent_path = '../Data/Cleaned Data/Independent_Data_dictionary.pkl'
dependent_path = '../Data/Cleaned Data/Dependent_Data_dictionary.pkl'

# Load the data
with open(independent_path, 'rb') as f:
    Independent_Data = pickle.load(f)

with open(dependent_path, 'rb') as f:
    Dependent_Data = pickle.load(f)

Encoding the dependent data

In [3]:
Dependent_Data_Encoded = {}

for name, df in Dependent_Data.items():
    # Creating a copy of the dataframe to modify
    encoded_df = df.copy()
    
    # Initialize the LabelEncoder
    le = LabelEncoder()
    
    # Iterate through each column in the dataframe
    for column in df.columns:
        # Encode the column if it is of type 'object' or 'category'
        if df[column].dtype == 'object' or df[column].dtype.name == 'category':
            # Fit label encoder and return encoded labels
            encoded_df[column] = le.fit_transform(df[column])
    
    # Add the encoded dataframe to the new dictionary
    Dependent_Data_Encoded[name] = encoded_df

# Now Dependent_Data_Encoded contains all the label-encoded dataframes


Encode the independent data

In [4]:
Independent_Data_Encoded = {}

for key, df in Independent_Data.items():
    df_encoded = df.copy()  # Create a copy of the dataframe to encode
    for column in df_encoded.columns:
        if df_encoded[column].dtype == 'object' or df_encoded[column].dtype.name == 'category':
            le = LabelEncoder()
            # Fit label encoder and return encoded labels
            df_encoded[column] = le.fit_transform(df_encoded[column])
    # Store the encoded dataframe in the new dictionary
    Independent_Data_Encoded[key] = df_encoded

# Now Independent_Data_Encoded contains all the encoded dataframes


Apply Cross Validation

In [5]:
# Initialize cross validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

#Initiate the model to normalize data
scaler = MinMaxScaler()

Breaking and Saving the Data

In [ ]:
# Suppress warnings
warnings.filterwarnings("ignore")

# Processing datasets
for dataset_name, X in Independent_Data_Encoded.items():
    # if dataset_name in simpler_data:

    try:
        fold_data = {}  # Dictionary to store fold data
        times = []
        print(f'Processing dataset: {dataset_name}')
        y = Dependent_Data_Encoded[dataset_name]

        fold_count = 1
        # Manually handling cross-validation
        for train_index, test_index in kf.split(X):
            fold_key = f'fold{fold_count}'
            fold_count += 1
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # Apply normalization on each DataFrame
            X_train_transformed = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
            X_test_transformed = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

            fold_data[fold_key] = {
                    'Training_Independent': X_train_transformed,
                    'Training_Dependent': y_train,
                    'Testing_Independent': X_test_transformed,
                    'Testing_Dependent': y_test
            }

        # Serialize fold data to a .pkl file
        with open(f'../Data/BASIC/{dataset_name}_fold_data.pkl', 'wb') as file:
            pickle.dump(fold_data, file)
    
    except Exception as e:
        print(f"Error processing {dataset_name}: {e}")
        traceback.print_exc()  # This will print the traceback of the exception

